In [1]:
import sys; sys.path.insert(0, "..")

import aoc

year, day = 2020, 20

puzzle = aoc.setup(year, day)
plines = puzzle.splitlines()

# Day 20

### Puzzle 1

In [2]:
import math


def edge_to_int(edge):
    x1 = int("".join(edge).replace(".", "0").replace("#", "1"), 2)
    x2 = int("".join(reversed(edge)).replace(".", "0").replace("#", "1"), 2)
    return min(x1, x2)
    
def solve1():
    m = {}
    for num, *lines in map(str.splitlines, puzzle.split("\n\n")):
        t = list(zip(*lines))
        num = int(num.split()[1][:-1])
        for e in map(edge_to_int, [lines[0], lines[-1], t[0], t[-1]]):
            m.setdefault(e, []).append(num)
    border = [v[0] for k, v in m.items() if len(v) == 1]
    return math.prod(set([x for x in border if border.count(x) == 2]))

solve1()

15670959891893

In [3]:
%timeit solve1()

2.52 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Puzzle 2

In [4]:
op = [1, 0, 3, 2]
monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.splitlines()


def rotflip(tile, o):
    c = (3, 1)
    k = True
    while c != o:
        if k:
            tile = list(zip(*tile))
            c = c[::-1]
        else:
            tile = [line[::-1] for line in tile]
            c = (op[c[0]], c[1])
        k = not k
    return list(map("".join, tile))


def solve2():
    m = {}
    tiles = {}
    for num, *lines in map(str.splitlines, puzzle.split("\n\n")):
        t = list(zip(*lines))
        num = int(num.split()[1][:-1])
        tiles[num] = [line[1:-1] for line in lines[1:-1]]
        for i, e in enumerate(map(edge_to_int, [lines[0], lines[-1], t[0], t[-1]])):
            m.setdefault(e, []).append((i, num))
    
    graph = {}
    for (o1, t1), (o2, t2) in [v for k, v in m.items() if len(v) == 2]:
        graph.setdefault(t1, {})[o1] = (o2, t2)
        graph.setdefault(t2, {})[o2] = (o1, t1)
    
    corner = next(k for k, v in graph.items() if len(v) == 2)
    border = {corner}
    dim = []
    for k in graph[corner]:
        c = corner
        x = 1
        while k in graph[c]:
            k, c = graph[c][k]
            k = op[k]
            border.add(c)
            x += 1
        dim.append(x)
    width, height = dim
    
    for k, v in graph.items():
        if k in border:
            continue
        for i in range(4):
            v.setdefault(i, None)
    
    tiled_image = [[None] * width for _ in range(height)]
    
    queue = [(corner, 0, 0, *graph[corner])]
    visited = set()
    while queue:
        p, i, j, a, b = queue.pop(0)
        
        if p in visited: continue
        visited.add(p)
        
        tiled_image[i][j] = rotflip(tiles[p], (a, b))
        
        for o1, u, v in [(a, i, j+1), (b, i+1, j)]:
            if graph[p].get(o1) is None:
                continue
            o2, q = graph[p][o1]
            if len(graph[q]) >= 3:
                graph[q].pop(o2)

            if len(graph[q]) <= 2:
                if o1 == a:
                    x = op[o2]
                    y, = set(graph[q]) - {x, o2}
                else:
                    y = op[o2]
                    x, = set(graph[q]) - {y, o2}
                queue.append((q, u, v, x, y))
    
    image = []
    for tile_line in tiled_image:
        image += ["".join(line) for line in zip(*tile_line)]
    
    monster_parts = set()
    for i in range(4):
        for j in range(4):
            if i in (j, op[j]):
                continue
            m = rotflip(monster, (i, j))
            for y in range(len(image) - len(m) + 1):
                for x in range(len(image[0]) - len(m[0]) + 1):
                    p = [(y+i, x+j) for i, line in enumerate(m) for j, c in enumerate(line) if c == "#"]
                    if all(image[i][j] == "#" for i, j in p):
                        monster_parts.update(p)
    return sum(line.count("#") for line in image) - len(monster_parts)


solve2()

1964

In [5]:
%timeit solve2()

652 ms ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
